#현재 epoch 6 버전이 가장 정확도 높음

> 0.9222222222222223

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import io
import binascii

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#이미지 불러오기

In [2]:
PATH = '/content/drive/MyDrive/Colab Notebooks/파이널프로젝트/data/농산물 품질(QC) 이미지'
os.getcwd()

'/content'

In [ ]:
folder_list = os.listdir(PATH+'/Validation_unzip')

In [ ]:
images = []
labels = []
for folder in folder_list:
  img_list = os.listdir(PATH + '/Validation_unzip/' + folder)
  for img in img_list[:10]:
    images.append(Image.open(PATH + '/Validation_unzip/' + folder +'/'+ img))
    labels.append(folder)

print(images[0])
print(labels[:10])

IndexError: ignored

#모델 불러오기

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/파이널프로젝트/data/saved_model/21_08_31_fruit_classification_model_V5_epo4')
model.summary()

# 예측하기

In [ ]:
from keras.preprocessing import image

x = image.img_to_array(images[0])
x = x.reshape((1,)+x.shape)


IndexError: ignored

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

batch = datagen.flow(x, batch_size=1)
plt.figure(0)
imgplot = plt.imshow(image.array_to_img(batch[0][0]))
plt.show()

pred = model.predict(batch)
print(pred)

NameError: ignored

In [ ]:
pred.argmax()

0

In [ ]:
folder_list[pred.argmax()]

'apple_fuji'

#리스트 한번에 예측하기

In [ ]:
folder_list.sort()
folder_list

['apple_fuji',
 'apple_yanggwang',
 'mandarine_hallabong',
 'mandarine_onjumilgam',
 'pear_chuhwang',
 'pear_singo',
 'persimmon_bansi',
 'persimmon_booyu',
 'persimmon_daebong']

In [ ]:
from keras.preprocessing import image

pred_list = []

for img in images:
  #img배열 3 -> 4차원으로 변경
  x = image.img_to_array(img)
  x = x.reshape((1,)+x.shape)
  
  datagen = ImageDataGenerator(rescale=1./255)
  batch = datagen.flow(x, batch_size=1)
  pred_list.append(model.predict(batch))

In [ ]:
correct = 0
wrong = 0
for idx in range(len(images)):
  i = pred_list[idx].argmax()
  plt.figure(idx)
  imgplot = plt.imshow(images[idx])
  print('=============================================================')
  plt.show()
  print('정답 :', labels[idx], '예측값 :', folder_list[i])
  if (labels[idx] == folder_list[i]):
    print('맞았습니다.')
    correct += 1
  else:
    print('틀렸습니다.')
    wrong += 1

  print("정확도: ", np.round(pred_list[idx], 2))
  print('=============================================================')
print(correct)
print(wrong)
print(correct / (correct+wrong))

Output hidden; open in https://colab.research.google.com to view.

#모델 비교

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/파이널프로젝트/data/saved_model/21_08_31_fruit_classification_model_V5_epo8')
model.summary()

from keras.preprocessing import image

pred_list = []

for img in images:
  #img배열 3 -> 4차원으로 변경
  x = image.img_to_array(img)
  x = x.reshape((1,)+x.shape)
  
  datagen = ImageDataGenerator(rescale=1./255)
  batch = datagen.flow(x, batch_size=1)
  pred_list.append(model.predict(batch))

correct = 0
wrong = 0
for idx in range(len(images)):
  i = pred_list[idx].argmax()
  plt.figure(idx)
  imgplot = plt.imshow(images[idx])
  print('=============================================================')
  plt.show()
  print('정답 :', labels[idx], '예측값 :', folder_list[i])
  if (labels[idx] == folder_list[i]):
    print('맞았습니다.')
    correct += 1
  else:
    print('틀렸습니다.')
    wrong += 1

  print("정확도: ", np.round(pred_list[idx], 2))
  print('=============================================================')
print(correct)
print(wrong)
print(correct / (correct+wrong))

Output hidden; open in https://colab.research.google.com to view.

#Validation Set으로 테스트

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

validation_dir = PATH +'/Validation_unzip'
validation_dir

validation_data = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(256, 256),
    batch_size= 100,
    class_mode='categorical'
)

model_1 = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/파이널프로젝트/data/saved_model/21_08_31_fruit_classification_model_V5_epo4')
model_2 = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/파이널프로젝트/data/saved_model/21_08_31_fruit_classification_model_V5_epo6')
model_3 = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/파이널프로젝트/data/saved_model/21_08_31_fruit_classification_model_V5_epo8')
model_3 = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/파이널프로젝트/data/saved_model/21_08_31_fruit_classification_model_V5_epo10')

Found 0 images belonging to 9 classes.


In [ ]:
loss, acc = model_1.evaluate(validation_data, verbose=2)

79/79 - 435s - loss: 0.2103 - accuracy: 0.9069


In [ ]:
loss, acc = model_2.evaluate(validation_data, verbose=2)

79/79 - 282s - loss: 0.2373 - accuracy: 0.9004


In [ ]:
loss, acc = model_3.evaluate(validation_data, verbose=2)

79/79 - 270s - loss: 0.1536 - accuracy: 0.9319


In [ ]:
loss, acc = model_4.evaluate(validation_data, verbose=2)

In [ ]:
import os 
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

model_3.save('/content/drive/MyDrive/Colab Notebooks/파이널프로젝트/data/saved_model/fruit_classification_model')

2.6.0
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/파이널프로젝트/data/saved_model/fruit_classification_model/assets
